<a href="https://colab.research.google.com/github/Cchrisekwugum/sqlite3_and_python_projects/blob/main/Scrape_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lets scrape from data from a website

The Department of Forestry and Fire Protection, California

In [ ]:
# import required libraries
import json  # parse json data from API response
import sqlite3  # interact with SQlite database
import urllib.request
import pandas as pd
import os
print(os.getcwd())

/content


In [ ]:
url = "https://www.fire.ca.gov/api/sitecore/Incident/GetFiresForMap?showFeatured=false"
headers = {"User-Agent": "Cafirehistorydb (chrisony4@gmail.com)"} # user agent identification

In [ ]:
# lets fetch the data...creeate an http request the url and header above
print("Fetching data from API...")
req = urllib.request.Request(url,headers = headers)
with urllib.request.urlopen(req) as response: # opens a connection to the API
    json_data = json.loads(response.read().decode("utf-8")) # read the response as jason data

Fetching data from API...


In [ ]:
#data = json.loads(json_data)
print(f"Received data for {len(json_data)} fire incidents")

Received data for 23 fire incidents


In [ ]:
# check to see if the data has been fetched
json_data[21:]

[{'Name': 'Quality Fire',
  'Updated': '2024-07-14',
  'Started': '2024-07-13',
  'AdminUnit': 'CAL FIRE San Bernardino Unit',
  'County': 'Inyo',
  'Location': 'Lone Pine Golf Course Road and Main Street',
  'AcresBurned': 51.0,
  'PercentContained': 50.0,
  'Longitude': -118.057587,
  'Latitude': 36.576457,
  'Type': None,
  'UniqueId': '8ff00a3e-f9a6-4c09-8aa0-5549f351312f',
  'Url': '/incidents/2024/7/13/quality-fire/',
  'StartedDateOnly': None,
  'IsActive': True,
  'ExternalUrl': ''},
 {'Name': 'Mcnab Fire',
  'Updated': '2024-07-14',
  'Started': '2024-07-14',
  'AdminUnit': 'CAL FIRE Mendocino Unit',
  'County': 'Mendocino',
  'Location': '800 Block of McNab Ranch Road, Ukiah',
  'AcresBurned': 10.0,
  'PercentContained': 0.0,
  'Longitude': -123.15297,
  'Latitude': 39.041985,
  'Type': None,
  'UniqueId': '085e88f6-33b7-4aef-adea-cda9dc7ef7f4',
  'Url': '/incidents/2024/7/14/mcnab-fire/',
  'StartedDateOnly': None,
  'IsActive': True,
  'ExternalUrl': ''}]

In [ ]:
# lets create a connection with SQLite database and create a database
# the script below is an SQL script for creating data tables and column names
with sqlite3.connect("cafires.db") as db:
    cursor = db.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS incidents (
        UniqueId TEXT PRIMARY KEY,
        Name TEXT,
        Updated TEXT,
        Started TEXT,
        AdminUnit TEXT,
        County TEXT,
        Location TEXT,
        AcresBurned REAL,
        PercentContained REAL,
        Longitude REAL,
        Latitude REAL,
        Url TEXT,
        IsActive INTEGER
    )
    """)


In [ ]:
# I will intialize a counter here and updates the records(or values as it is called in pandas)
new_update_count = 0
for item in json_data:
  cursor.execute("""
                 INSERT OR REPLACE INTO incidents (
                     UniqueId,Name, Updated,Started,AdminUnit,County,Location,AcresBurned,PercentContained,Longitude,Latitude,Url,IsActive
                 ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
                 """,
                  (
            item["UniqueId"],
            item["Name"],
            item["Updated"],
            item["Started"],
            item["AdminUnit"],
            item["County"],
            item["Location"],
            item["AcresBurned"],
            item["PercentContained"],
            item["Longitude"],
            item["Latitude"],
            item["Url"],
            1 if item["IsActive"] else 0,
        ),
          )
  if cursor.rowcount > 0:
        new_update_count +=  cursor.rowcount
        db.commit()  # Commit changes to the database

        print(f"Inserted or updated: {item['Name']} (ID: {item['UniqueId']})")

print(f"Total rows inserted or updated: {new_update_count}")
print("Operation completed...")


Inserted or updated: Basin Fire (ID: 9226f0ee-ae1d-4ca7-8a21-9390b6215c85)
Inserted or updated: Sheering Fire (ID: 4d1eb95c-6e27-4af4-9133-d053b8310cbd)
Inserted or updated: Shelly Fire (ID: 798bb0b3-8d70-4d9c-a0b6-094c9013a4da)
Inserted or updated: Lake Fire (ID: bc6db02b-254b-4857-9bd9-2ec52580960e)
Inserted or updated: Vista Fire (ID: 3a6eccd0-40fa-4d68-97fa-e38f93f0458c)
Inserted or updated: North Fire (ID: f2edef16-123c-47df-8cfb-5822aaa4e6eb)
Inserted or updated: Royal Fire (ID: fb03fcdb-e84c-4edd-8457-126b805119f2)
Inserted or updated: Mina Fire (ID: d08b8b6b-585b-40c2-9b40-99ec5adf5513)
Inserted or updated: Bogus Fire (ID: 1d2f4147-5768-4f7a-b777-42f72c9d75b4)
Inserted or updated: Cow Fire (ID: 704f4c46-c67f-4cd6-ad76-1bb893764f0e)
Inserted or updated: Point Fire (ID: fdb9b539-dbfe-4112-98b3-ce893d831e9e)
Inserted or updated: Pauba Fire (ID: 6ba3ef43-a6ad-486d-8df8-ec24dfdcbfff)
Inserted or updated: Hurricane Fire (ID: 5d90917c-38cf-4ea9-86d6-e3a1d2728b0a)
Inserted or updated: 

In [ ]:
conn = sqlite3.connect('cafires.db')
df = pd.read_sql_query("SELECT * FROM incidents", conn)
conn.close()

In [ ]:
df.head(2)

,UniqueId,Name,Updated,Started,AdminUnit,County,Location,AcresBurned,PercentContained,Longitude,Latitude,Url,IsActive
0,9226f0ee-ae1d-4ca7-8a21-9390b6215c85,Basin Fire,2024-07-11,2024-06-26,Sierra National Forest Service,Fresno,North of Green Cabin Flat Campground,14023.0,95.0,-119.110336,36.868064,/incidents/2024/6/26/basin-fire/,1
1,4d1eb95c-6e27-4af4-9133-d053b8310cbd,Sheering Fire,2024-07-10,2024-06-24,Stanislaus National Forest,Tuolumne,East of Bourland Creek & North of Lost Creek,528.0,89.0,-119.985167,38.163167,/incidents/2024/6/24/sheering-fire/,1


In [ ]:
# save the sql_query to csv file
df.to_csv('cafires.csv',index = False)

In [ ]:
# import it back as a csv file
df = pd.read_csv('cafires.csv')

In [ ]:
df.head(2)

,UniqueId,Name,Updated,Started,AdminUnit,County,Location,AcresBurned,PercentContained,Longitude,Latitude,Url,IsActive
0,9226f0ee-ae1d-4ca7-8a21-9390b6215c85,Basin Fire,2024-07-11,2024-06-26,Sierra National Forest Service,Fresno,North of Green Cabin Flat Campground,14023.0,95.0,-119.110336,36.868064,/incidents/2024/6/26/basin-fire/,1
1,4d1eb95c-6e27-4af4-9133-d053b8310cbd,Sheering Fire,2024-07-10,2024-06-24,Stanislaus National Forest,Tuolumne,East of Bourland Creek & North of Lost Creek,528.0,89.0,-119.985167,38.163167,/incidents/2024/6/24/sheering-fire/,1


In [ ]:
df.isna().sum()

UniqueId            0
Name                0
Updated             0
Started             0
AdminUnit           0
County              0
Location            0
AcresBurned         0
PercentContained    0
Longitude           0
Latitude            0
Url                 0
IsActive            0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.County.value_counts()

County
Tulare             3
Siskiyou           2
Mendocino          2
San Luis Obispo    2
Kern               2
Lassen             2
Fresno             1
Tuolumne           1
Santa Barbara      1
San Bernardino     1
Modoc              1
Placer             1
Humboldt           1
Riverside          1
Los Angeles        1
Inyo               1
Name: count, dtype: int64

In [ ]:
import openpyxl

In [ ]:
from openpyxl import load_workbook, Workbook